### Import Geopy modules for geo-coding

In [1]:
#geopy imports 
from geopy.extra.rate_limiter import RateLimiter
from geopy import Nominatim
locator = Nominatim(user_agent="myGeocoder")
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

### Import Folium modules for mapping

In [33]:
#folium imports
import folium
from folium import Map
from folium.plugins import HeatMap

### Set Jupyter Notebook configs

In [3]:
#set jp options
pd.set_option('display.max_colwidth', 300)
pd.set_option('display.max_rows', 900)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Import dataset and munge for use in mapping

In [38]:
vets = pd.read_csv('../data/vets_SOF.csv',  encoding="ISO-8859-1")
vets['Location'] = vets['City'] + ', ' + vets['State']

In [56]:
data = vets['Location'].value_counts().rename_axis('Location').reset_index(name='Counts')

#remove artifacts
data = data[data['Location'] != 'OCONUS, OCONUS']
data = data[data['Location'] != 'None , None']
data = data[data['Location'] != 'Si, CO']

In [59]:
#check to ensure that I have the data in the correct Location, Count format
data.head()

,Location,Counts
0,"San Diego, CA",92
1,"Virginia Beach, VA",79
2,"USA, USA",32
3,"Washington, DC",30
4,"New York, NY",26


In [60]:
#convert physical locations to coordinates
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
data['Coordinates'] = data['Location'].apply(geocode)

#convert coordinates to individual lat and longs
data['point'] = data['Coordinates'].apply(lambda loc: tuple(loc.point) if loc else None)
data[['latitude', 'longitude', 'altitude']] = pd.DataFrame(data['point'].tolist(), index=data.index)

In [66]:
#ensure that I have the points in the right place
data.head()

,Location,Counts,Coordinates,point,latitude,longitude,altitude
0,"San Diego, CA",92,"(San Diego, San Diego County, California, United States of America, (32.7174209, -117.1627714))","(32.7174209, -117.1627714, 0.0)",32.717421,-117.162771,0.0
1,"Virginia Beach, VA",79,"(Virginia Beach, Virginia Beach (city), Virginia, United States of America, (36.8529841, -75.9774183))","(36.8529841, -75.9774183, 0.0)",36.852984,-75.977418,0.0
2,"USA, USA",32,"(United States, (39.7837304, -100.4458825))","(39.7837304, -100.4458825, 0.0)",39.783730,-100.445882,0.0
3,"Washington, DC",30,"(Washington, District of Columbia, 20230, United States of America, (38.8949924, -77.0365581))","(38.8949924, -77.0365581, 0.0)",38.894992,-77.036558,0.0
4,"New York, NY",26,"(New York, United States of America, (40.7127281, -74.0060152))","(40.7127281, -74.0060152, 0.0)",40.712728,-74.006015,0.0


In [68]:
#standardize weights to between 0 and 1
data['Counts'] = data['Counts']/max(data['Counts'])

#finalize data structure for input into heatmap
final_data = data[['latitude', 'longitude', 'Counts']].values

### Create base map and HeatMap

In [72]:
#get base coordinates of USA
base = locator.geocode('USA')
base_coords = base[-1]

#initialize base map
base_map = Map(location=base_coords, control_scale=True, zoom_start=5)

#generate HeatMap
HeatMap(final_data, min_opacity=1, radius=9).add_to(base_map)
base_map

In [73]:
data.to_csv('../data/cleaned_data.csv')

In [74]:
vets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 754 entries, 0 to 753
Data columns (total 32 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Unnamed: 0                              754 non-null    int64  
 1   Id                                      754 non-null    object 
 2   Birthdate                               78 non-null     object 
 3   CreatedDate                             754 non-null    object 
 4   Location                                754 non-null    object 
 5   City                                    754 non-null    object 
 6   State                                   754 non-null    object 
 7   How_did_you_hear_about_Elite_Meet       542 non-null    object 
 8   Your_transition_to_civilian_life_begin  453 non-null    object 
 9   Undergrad                               640 non-null    object 
 10  Undergraduate_School                    687 non-null    object